In [1]:
from models.autoencoder import AutoEncoder
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms
import torch

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((160, 160))
])
celeb = torchvision.datasets.CelebA('data/', download=False, transform=transform)
data_loader = DataLoader(celeb,
                          batch_size=64,
                          shuffle=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
model = AutoEncoder(160, 3, 512)
model

AutoEncoder(
  (encoder): Encoder(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (fc): Linear(in_features=51200, out_features=512, bias=True)
  )
  (decoder): Decoder(
    (fc): Linear(in_features=512, out_features=51200, bias=True)
    (deconv1): ConvTranspose2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (deconv2): ConvTranspose2d(128, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (deconv3): ConvTranspose2d(64, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (conv1): Conv2d(32, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
)

In [7]:
model(next(iter(data_loader))[0]).shape

torch.Size([64, 512])

In [8]:
import lightning as pl

trainer = pl.Trainer()
trainer.fit(model, data_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type    | Params
------------------------------------
0 | encoder | Encoder | 26.3 M
1 | decoder | Decoder | 26.5 M
------------------------------------
52.8 M    Trainable params
0         Non-trainable params
52.8 M    Total params
211.258   Total estimated model params size (MB)


Training: |                                                                                                   …

In [12]:
data = next(iter(data_loader))
data[0], data[1] = data[0].to(device), data[1].to(device)
model.training_step(batch=data, batch_idx=1)

MisconfigurationException: You are trying to `self.log()` but it is not managed by the `Trainer` control flow